In [2]:
import torch
from tqdm import tqdm
torch.cuda.empty_cache() # PyTorch thing

In [3]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import PeftConfig
from tqdm import tqdm
import re

model_id = "Open-Orca/Mistral-7B-OpenOrca"
adapter_model_id = "/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/output/checkpoint-html-pipe-250"
tokenizer = AutoTokenizer.from_pretrained(model_id)


/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Special tokens have been added in the vocabulary, make sure the associated word embeddin

In [4]:
import torch
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
peft_config = PeftConfig.from_pretrained(adapter_model_id)

In [ ]:
# to initiate with random weights
peft_config.init_lora_weights = False

model.add_adapter(peft_config)
model.enable_adapters()

In [ ]:
import pandas as pd
df = pd.read_csv('../data/pipe_encoded_convfinqa.csv')

In [ ]:
row = df.loc[17166]

In [ ]:
# Function to add backticks to HTML tags
def add_backticks_to_html_tags(html_content):
    # Define the regular expression pattern to match HTML tags
    pattern = r'<([^>]+)>'

    # Replace HTML tags with backticks added
    return re.sub(pattern, r'`<\1>`', html_content)

In [ ]:
def prompt_translation(row):
    eval_mode = True
    addon_prompt = "Read the following texts and table with financial data from an S&P 500 earnings report carefully.Based on the question-answer history (if provided), answer the last question. The answer may require mathematical calculation based on the data provided."
    input_prompt =f"""<s> [INST] {addon_prompt}\n
        {add_backticks_to_html_tags(row['PIPE_encoding'])}\n
        {row['question']}\n
        [/INST]
        {row['answer']} </s>
        """
    return input_prompt

In [ ]:
df.columns

Index(['index', 'question', 'answer', 'PIPE_encoding'], dtype='object')

In [ ]:
output_df = pd.DataFrame(columns=['index', 'question', 'answer', 'prediction'])

In [ ]:
for i, row in tqdm(df.iterrows()):
    text = prompt_translation(row)
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}
    output = model.generate(**inputs)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extract values after [/INST]
    inst_values = decoded_output.split('[/INST]')[1].strip()
    output_df.at[i, 'index'] = row['index']
    output_df.at[i, 'question'] = row['question']
    output_df.at[i, 'answer'] = row['answer']
    output_df.at[i, 'prediction'] = inst_values



0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 174, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids 

In [ ]:
output_df.to_csv('../output/checkpoint_html_pipe_250.csv', index=False)